#Loading google/gemma-2b

In [ ]:
!pip install optimum
!pip install peft
!pip install datasets

In [ ]:
!pip install accelerate
!pip install --upgrade bitsandbytes

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
from datasets import load_dataset
import torch
import transformers

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(load_in_4bit=True)  #for loading model in 4 bit quantized format
hf_token = 'your_hg_token'

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b", token = hf_token,
                                          use_fast = True)
model = AutoModelForCausalLM.from_pretrained("google/gemma-2b",token = hf_token, quantization_config=quantization_config)

#model size = 4.95G

tokenizer_config.json:   0%|          | 0.00/33.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [ ]:
#eval state for dropouts to be deactivated
model.eval()
prompt = "Hey, I'm thinking of investing in the stock market, but it all seems so daunting. Any advice for a beginner?"

inputs = tokenizer(prompt,
                   return_tensors = "pt")

outputs = model.generate(input_ids = inputs['input_ids'],
                         max_new_tokens = 150)

print(f'Model Output:\n  {tokenizer.batch_decode(outputs)[0]}')

Model Output:
  <bos>Hey, I'm thinking of investing in the stock market, but it all seems so daunting. Any advice for a beginner?
I've been investing in the stock market for over 10 years now. I've been a pretty good investor, and have made a pretty good return.

I've been investing in the stock market since I was 1


## Improving model by finetuning with QLoRA

In [ ]:
model.train()  #dropouts acctivated

model.gradient_checkpointing_enable()  #memory saving technique that clears specific activations and recomputes during backward pass

#enable quantized training
model = prepare_model_for_kbit_training(model)  #initial model is in 4 bit precision and training on lora will be high precision


In [ ]:
def find_target_modules_for_lora(model):

  layers = set()
  for name, module in model.named_modules():
    if "Linear" in str(type(module)):
      layer_type = name.split('.')[-1]
      layers.add(layer_type)

  return list(layers)

target_modules = find_target_modules_for_lora(model)
target_modules

['down_proj',
 'q_proj',
 'gate_proj',
 'o_proj',
 'up_proj',
 'k_proj',
 'v_proj',
 'lm_head']

In [ ]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    (print(f'Number of trainable parameters: {trainable_model_params}'))
    return trainable_model_params

In [ ]:
lora_config = LoraConfig(
    r = 4,   #rank
    lora_alpha = 32,
    target_modules = ["q_proj"],
    lora_dropout = 0.05,
    bias = "none",
    task_type = "CAUSAL_LM"
)

model = get_peft_model(model, lora_config)  #LoRA trainable model


In [ ]:
dataset = load_dataset('innova-ai/Human-Style-Answers')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/931 [00:00<?, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 931
    })
})

In [ ]:
dataset['train']['text'][0]

"### Human: Hey, I'm thinking of investing in the stock market, but it all seems so daunting. Any advice for a beginner? 😕\n\n### Assistant: Absolutely, my friend! Investing in the stock market can be a great way to build wealth over time. Here's a simple breakdown to get you started: First, determine your investment goals. Are you saving for retirement, a down payment on a house, or something else? This will help guide your strategy. Next, educate yourself. Read up on basic investment concepts, understand the risks and potential rewards, and learn about different types of stocks and how the market works. Then, open a brokerage account. There are many online brokers to choose from, offering various features and fees, so compare a few before deciding. Start small and diversify. Don't put all your money into one stock; instead, invest in a variety of companies and industries to minimize risk. And finally, think long-term. The stock market can be volatile, with ups and downs, but staying 

In [ ]:
def tokenize_data(examples):
  text = examples['text']

  tokenizer.truncation_side = "left"

  tokenized_inputs = tokenizer(
      text,
      return_tensors = 'np',  #returns numpy tensors
      truncation = True,  #truncation
      max_length = 250 #max text length for trncation
  )

  return tokenized_inputs

tokenized_data = dataset.map(tokenize_data, batched = True)  #applyin tokenize function

Map:   0%|          | 0/931 [00:00<?, ? examples/s]

In [ ]:
import transformers
#batched data when data in batch is of different legth, that can be resolved by data collator by dynamically padding them to make them same length
tokenizer.pad_token = tokenizer.eos_token

#data collator
data_collator = transformers.DataCollatorForLanguageModeling(tokenizer, mlm = False)

In [ ]:
 #hyper params for trainer

training_args = transformers.TrainingArguments(
    output_dir = 'qlora-ft',
    learning_rate = 2e-4,
    per_device_train_batch_size = 4,
    num_train_epochs = 1,
    weight_decay = 0.01,
    logging_strategy = "epoch",
    save_strategy = "epoch",
    gradient_accumulation_steps = 4,
    warmup_steps = 2,
    optim = "paged_adamw_8bit",  #paged optimizer for paged optimization
    logging_steps = 1,
)

In [ ]:
trainer = transformers.Trainer(
    model = model,
    train_dataset = tokenized_data['train'],
    args = training_args,
    data_collator = data_collator
)

model.config.use_cache = False  #silence warnings

#train model
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
58,2.435000


/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:581: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-66323767-0b09c5f06d6ba32f16fdb982;a2ae1d83-dcbe-488e-88f9-07b7adf6fa56)

Cannot access gated repo for url https://huggingface.co/google/gemma-2b/resolve/main/config.json.
Access to model google/gemma-2b is restricted. You must be authenticated to access it. - silently ignoring the lookup for the file config.json in google/gemma-2b.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in google/gemma-2b - will assume that the vocabulary was not modified.
  warnings.warn(


TrainOutput(global_step=58, training_loss=2.4350172897865034, metrics={'train_runtime': 745.4213, 'train_samples_per_second': 1.249, 'train_steps_per_second': 0.078, 'total_flos': 2027912169455616.0, 'train_loss': 2.4350172897865034, 'epoch': 0.9957081545064378})

In [ ]:
model.eval()
prompt = "I'm a beginner, and I'm worried about losing all my money. How can I protect myself?. Any advice??"

inputs = tokenizer(prompt,
                   return_tensors = "pt")

outputs = model.generate(input_ids = inputs['input_ids'],
                         max_new_tokens = 70)

print(f'Model Output:\n  {tokenizer.batch_decode(outputs)[0]}')

Model Output:
  <bos>I'm a beginner, and I'm worried about losing all my money. How can I protect myself?. Any advice??

I've been playing for a few months now, and I've been pretty lucky. I've only lost a few bucks, and I've been able to win a few bucks too.

I think the key is to not get too greedy. If you win, great! But don't go out and spend all your
